In [7]:
if(grepl("kaggle",getwd())){
    paths <- list(
        data = file.path("/kaggle", "input", "gfi-iran"),
        output = file.path("/kaggle", "working","output"),
        temp = file.path("/kaggle", "working","intermediate-data")
    )
}else{
    paths <- list(
        data = file.path("..","data"),
        output = file.path("..","output"),
        temp = file.path("..","intermediate-data")
    )
    for(i in list.files(paths$data,full.names = TRUE)){
        unzip(i,exdir = paths$data)
    }
    file.rename(paths$data |> file.path("IRN_2017_FINDEX_v02_M_EXCEL\\micro_irn_varlabel.xls"),
    paths$data |> file.path("micro_irn_varlabel.xls"))
    #file.rename(paths$data |> file.path("IRN_2017_FINDEX_v02_M_EXCEL\\micro_irn_varname.xls"),
    #paths$data |> file.path("micro_irn_varname.xls"))
}

Warning message in unzip(i, exdir = paths$data):
“error 1 in extracting from zip file”
Warning message in unzip(i, exdir = paths$data):
“error 1 in extracting from zip file”
Warning message in unzip(i, exdir = paths$data):
“error 1 in extracting from zip file”
Warning message in unzip(i, exdir = paths$data):
“error 1 in extracting from zip file”


[1] TRUE

In [8]:
# Define a vector of package names to be used in the script
package_names_vec <- c(
    "tidyverse",
    "ggtext",
    "showtext",
    "tidymodels",
    "furrr",
    "readxl",
    "tictoc",
    "glmnet",
    "readxl"
)

# Define a function to install and load required packages
package_prep <- function(names_vec) {
  # Iterate through each package name in the provided vector
  for (name in names_vec) {
    # Check if the package is already installed
    if (!require(name, character.only = TRUE)) {
      # Install the package using renv if not already installed
      install.packages(name)
    }
  }
}

# Pipe the vector of package names into the package_prep function
package_names_vec |> package_prep()


# Load the data and clean

In [9]:
# Define a function to load and process a table
# Parameters:
# - name: Name of the table to be assigned in the global environment
# - path: Path to the file to be read
# - argyear: Year to be added as a column in the resulting table
# - reading_func: Function to read the file (e.g., read_csv, read_excel)
load_tbl <- function(name, path, argyear, reading_func) {
  # Read the file, process it, and store it in a variable 'result'
  result <- file.path(path) |>  # Generate the full file path
    reading_func() |>           # Read the file using the provided reading function
    mutate(
      year = argyear,           # Add a 'year' column with the specified year
      ID = row_number()         # Add an 'ID' column with sequential row numbers
    ) |> 
    select(where(~!all(is.na(.x)))) # Select only the columns that are not entirely NA
  
  # Assign the processed table to the specified name in the global environment
  assign(name, result, envir = .GlobalEnv)
}

# Call the 'load_tbl' function to load and process data for the year 2021
load_tbl("gfi_2021_tbl",               # Name of the output table
  file.path(paths$data, "micro_irn.csv"),  # File path
  2021,                               # Year to assign
  read_csv                            # Reading function
)

# Call the 'load_tbl' function to load and process data for the year 2017 (variable labels)
load_tbl("gfi_2017_tbl",            # Name of the output table
  file.path(paths$data, "micro_irn_varlabel.xls"),  # File path
  2017,                               # Year to assign
  read_excel                          # Reading function
)

Rows: 1005 Columns: 84
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): economy, economycode
dbl (78): wpid_random, wgt, female, age, educ, inc_q, emp_in, account, accou...
lgl  (4): urbanicity_f2f, receive_agriculture, remittances, merchantpay_dig

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [10]:
gfi_2017_selected_tbl

id,year,weight,respondent_age,respondent_education_level,household_income_quintile,in_the_workforce,main_source_emergency_funds,paid_utility_bill,payments_utility_bills,has_account,has_financial_institution_account,has_mobile_money_account,borrowed_in_past_year,borrowed_from_financial_institution,payments_wage_payments,received_government_transfers,received_government_pension,received_government_pension_payment
<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2,2017,0.6743416,42,completed primary or less,Poorest 20%,out of workforce,NA,yes,1,yes,yes,0,yes,yes,"Main source: Borrowing from a bank, empoyer, or private lender",yes,no,4
4,2017,0.9317940,35,secondary,Middle 20%,out of workforce,NA,yes,1,yes,yes,0,yes,no,"Main source: Borrowing from a bank, empoyer, or private lender",yes,no,4
8,2017,1.2513574,37,completed primary or less,Middle 20%,out of workforce,NA,no,4,yes,yes,0,yes,yes,"Main source: Borrowing from a bank, empoyer, or private lender",yes,no,4
9,2017,1.9209096,20,secondary,Middle 20%,out of workforce,Main source: Family or friends,no,4,yes,yes,0,yes,no,"Main source: Borrowing from a bank, empoyer, or private lender",yes,no,4
11,2017,0.5956590,40,secondary,Fourth 20%,out of workforce,NA,yes,1,yes,yes,0,yes,yes,"Main source: Borrowing from a bank, empoyer, or private lender",yes,no,4
13,2017,0.9954276,76,completed tertiary or more,Second 20%,out of workforce,"Main source: Borrowing from a bank, empoyer, or private lender",yes,1,yes,yes,0,0,no,"Main source: Borrowing from a bank, empoyer, or private lender",no,no,4
14,2017,1.0036016,48,completed primary or less,Fourth 20%,in workforce,NA,yes,1,yes,yes,0,yes,yes,"Main source: Borrowing from a bank, empoyer, or private lender",yes,no,4
16,2017,0.3924855,37,secondary,Richest 20%,in workforce,NA,yes,2,yes,yes,yes,yes,yes,"Main source: Borrowing from a bank, empoyer, or private lender",yes,no,4
18,2017,1.9986920,32,completed primary or less,Poorest 20%,out of workforce,NA,yes,2,yes,yes,0,yes,no,"Main source: Borrowing from a bank, empoyer, or private lender",yes,no,4


In [ ]:
#gfi_2021_selected_tbl <- 
gfi_2021_tbl |> 
  filter(female ==1 ) |>
  select(year, ID, wgt, age, educ, emp_in, inc_q, fin24, pay_utilities,
  account, account_fin, account_mob, fin22a, borrowed, anydigpayment,
    fin32,fin38, fin30, receive_transfers, receive_pension) |>
  rename(
    id                               = ID,
    weight                           = wgt,
    respondent_age                   = age,
    respondent_education_level       = educ,
    household_income_quintile        = inc_q,
    in_the_workforce                 = emp_in,
    main_source_emergency_funds      = fin24,
    payments_utility_bills           = pay_utilities,
    has_account                      = account,
    has_financial_institution_account= account_fin,
    has_mobile_money_account         = account_mob,
    borrowed_in_past_year            = borrowed,
    borrowed_from_financial_institution = fin22a,
    made_or_received_digital_payment = anydigpayment,
    payments_wage_payments           = fin32,
    received_government_pension      = fin38,
    paid_utility_bill                = fin30, 
    received_government_transfers    = receive_transfers,
    received_government_pension_payment      = receive_pension
  ) |> 
  mutate(
    
  ) |> map(~unique(.x))

$year
[1] 2021

$id
  [1]    3    6    7    8    9   12   13   14   15   17   18   19   20   21   22
 [16]   24   25   26   27   29   31   34   36   37   40   42   43   45   46   47
 [31]   48   51   52   55   56   59   61   65   66   70   71   72   73   74   76
 [46]   77   80   86   88   90   92   93   96   97   98  100  103  106  107  109
 [61]  111  112  114  122  123  124  126  127  135  137  139  140  143  145  146
 [76]  151  154  155  156  157  159  160  161  163  165  166  168  169  170  171
 [91]  172  178  179  183  185  186  187  188  190  194  195  199  201  206  209
[106]  210  212  213  217  218  226  229  230  231  232  233  234  236  237  238
[121]  240  241  242  249  250  251  253  254  255  263  264  267  268  269  272
[136]  275  276  277  278  279  281  282  285  289  290  291  292  293  297  299
[151]  300  301  302  303  304  306  309  310  311  314  319  320  321  323  327
[166]  329  330  333  341  342  345  346  350  352  353  354  356  357  360  362
[181]  364  365  367  368  369  370  372  373  375  376  377  379  380  383  390
[196]  393  394  395  398  399  401  402  403  404  405  407  410  411  413  415
[211]  417  427  428  429  431  432  433  434  435  437  439  440  441  442  444
[226]  445  446  447  448  451  453  455  457  458  461  462  463  464  467  468
[241]  471  473  482  484  486  487  489  490  491  492  493  494  496  497  498
[256]  501  502  504  506  509  511  512  513  516  518  521  522  523  524  525
[271]  526  528  529  530  532  535  538  539  540  542  543  544  545  546  549
[286]  550  554  555  556  558  559  562  565  566  569  572  574  575  576  578
[301]  579  581  582  583  584  585  586  589  593  594  595  601  602  603  605
[316]  606  608  610  613  618  619  624  625  626  627  630  631  632  633  637
[331]  640  642  644  650  651  653  655  657  658  660  661  663  667  668  669
[346]  670  671  673  674  675  678  679  680  681  684  685  687  688  692  693
[361]  694  697  699  700  703  704  706  707  709  710  711  712  713  715  716
[376]  721  723  724  726  730  731  732  733  735  745  746  747  748  750  751
[391]  752  753  754  756  757  763  766  768  770  772  773  775  777  778  783
[406]  784  787  793  794  795  798  799  800  801  802  804  805  806  807  808
[421]  812  813  814  817  818  819  820  823  825  828  831  835  839  840  843
[436]  844  848  849  850  851  852  854  856  859  861  863  865  867  869  870
[451]  872  873  875  877  879  880  881  883  884  885  886  888  889  892  893
[466]  897  899  901  902  903  905  907  909  913  914  917  921  922  927  928
[481]  930  933  934  936  938  940  941  942  944  945  948  949  951  954  955
[496]  956  958  959  960  965  967  973  974  975  976  977  981  982  983  987
[511]  988  997  998  999 1000 1001 1002 1005

$weight
  [1] 1.4529222 0.6631506 1.2162853 0.9067121 0.9576944 0.8074431 1.7824843
  [8] 0.4929144 0.5565068 1.0091398 0.5854931 0.3043638 0.5620985 0.5531554
 [15] 1.2207564 0.8671475 1.1284994 0.3830186 0.7616278 0.7352495 1.8935686
 [22] 0.4839169 1.4642259 1.4595700 0.5498639 0.9519647 1.0850542 0.6489485
 [29] 1.0813260 0.4971166 1.2295707 0.7447657 0.9499451 1.1638567 1.3629371
 [36] 0.7434636 0.9912752 0.4079492 0.5679225 0.6734607 0.8866001 0.7994217
 [43] 1.3519170 0.6820527 0.7558466 0.7371003 0.9433616 0.4754205 0.4373954
 [50] 1.1139845 0.7860656 0.7909890 0.5475706 1.0999203 1.0991563 0.4988143
 [57] 0.9542125 0.4870403 1.2770297 0.7279035 0.8520191 0.7922092 0.3370236
 [64] 1.4151434 1.0570296 1.1156002 0.5153776 0.9492070 1.2660428 0.6355249
 [71] 0.8496275 0.6133334 0.5876117 0.6471927 1.2115410 1.2399844 0.9850280
 [78] 2.8434808 0.9576632 0.5718218 0.5603629 1.1509747 0.9169973 1.1621640
 [85] 0.7598437 0.7051206 0.8420577 0.9738363 0.5698983 1.0135192 0.4625599
 [92] 0.4858442 0.9678892 0.5685456 2.0096478 1.3411887 0.4663869 0.5725700
 [99] 1.4050817 0.9099665 1.4200230 0.8340964 0.6807874 1.6496954 0.7715516
[106] 1.0900841 0.8906012 1.061

In [ ]:
#gfi_2017_selected_tbl <- 
gfi_2017_tbl |>
  filter(`Respondent is female` == "Female") |>
  select(
    ID, year, Weight, `Respondent age`, `Respondent education level`,
    `Within-economy household income quintile`,
    `Respondent is in the workforce`,
    `Main source of emergency funds`, `Paid utility bills in past 12 months`,
    `Payments: utility bills`, `Has an account`,
    `Has an account at a financial institution`, `Has a mobile money account`,
    `Borrowed in the past year`,
    `Borrowed in past 12 months: from a financial institution`,
    `Payments: wage payments`,
    `Received government transfers in past 12 months`,
    `Received government transfers in past 12 months: pension`,
    `Payments: government pension`
  ) |>
  rename(
    id                                = ID,
    weight                            = Weight,
    respondent_age                    = `Respondent age`,
    respondent_education_level        = `Respondent education level`,
    household_income_quintile         = `Within-economy household income quintile`,
    in_the_workforce                  = `Respondent is in the workforce`,
    main_source_emergency_funds       = `Main source of emergency funds`,
   payments_utility_bills            = `Payments: utility bills`,
    paid_utility_bill                 = `Paid utility bills in past 12 months`,
    has_account                       = `Has an account`,
    has_financial_institution_account = `Has an account at a financial institution`,
    has_mobile_money_account          = `Has a mobile money account`,
    borrowed_in_past_year             = `Borrowed in the past year`,
    borrowed_from_financial_institution = `Borrowed in past 12 months: from a financial institution`,
    payments_wage_payments            = `Payments: wage payments`,
    received_government_transfers     = `Received government transfers in past 12 months`,
    received_government_pension       = `Received government transfers in past 12 months: pension`,
    received_government_pension_payment = `Payments: government pension`
  ) |> mutate(
    respondent_education_level = respondent_education_level |> case_match(
      c('(rf)','(dk)') ~ NA,
      'completed primary or less' ~ 0,
      'secondary' ~ 1,
      'completed tertiary or more' ~ 2
    ),
    household_income_quintile = household_income_quintile |> case_match(
      'Poorest 20%' ~ 0,
      'Middle 20%' ~ 1,
      'Fourth 20%' ~ 2,
      'Second 20%' ~ 3,
      'Richest 20%' ~ 4
    ),
    in_the_workforce = in_the_workforce |> case_match(
    'out of workforce' ~ 0,
    'in workforce' ~ 1
    ),
    main_source_emergency_funds = main_source_emergency_funds |> case_match(
      'Main source: Family or friends' ~ 0,
      'Main source: Borrowing from a bank, empoyer, or private lender' ~ 1,
      'Main source: Savings' ~ 2,
      'Main source: Money from working' ~ 3,
      'Main source: Selling assets' ~ 4,
      .default = NA
    )
  ) |> map(~unique(.x))

ERROR: Error in parse(text = input): <text>:45:20: unexpected string constant
44:     household_income_quintile = household_income_quintile |> case_match(
45:       'Poorest 20%''Middle 20%'
                       ^


In [8]:
long_gfi_2017_tbl <- gfi_2017_renamed_tbl |> 
  # Start with the renamed dataset and apply transformations
  
  # Remove unnecessary columns for further analysis
  select(-c(Economy, Economy_Code, Gallup_World_Poll_identifier)) |> 
  
  # Mutate to clean and standardize categorical variables
  mutate(
    Respondent_education_level = case_match(
      Respondent_education_level,
      c("(dk)", "(rf)") ~ NA, # Replace "don't know" or "refused" responses with NA
      .default = Respondent_education_level
    ),
    Main_source_of_emergency_funds = case_match(
      Main_source_of_emergency_funds,
      c("(dk)", "ref") ~ NA, # Similarly, standardize "don't know" or "refused" responses as NA
      .default = Main_source_of_emergency_funds
    )
  ) |> 
  
  # Reshape data from wide to long format
  pivot_longer(
    cols = -c(!!!syms(control_cols2017)), # Keep control variables static and reshape other columns
    names_to = "Question", # Name for reshaped variable columns
    values_to = "Answers" # Name for reshaped value columns
  ) |> 
  
  # Standardize and categorize answers
  mutate(
    Answers = case_match(
      Answers,
      c("yes", "Yes", "Possible") ~ "1", # Group affirmative responses under "1"
      c("no", "No", "Not possible") ~ "2", # Group negative responses under "2"
      c("(DK)", "(dk)") ~ "3", # Code "don't know" as "3"
      c("(ref)", "ref", "(Refused)") ~ "4", # Code "refused" responses as "4"
      .default = as.character(Answers) # Keep other responses as characters
    )
  )  |> 
  
  # Convert "Answers" to a factor for analysis
  mutate(Answers = fct(Answers)) |> 
  
  # Group by ID to prepare for question numbering
  group_by(ID) |> 
  mutate(Question_no = row_number()) |> # Assign a unique number to each question within each group
  ungroup() |> 
  
  # Apply transformations across multiple columns
  mutate(
    across(
      -c( # Exclude numeric or critical variables from factor conversion
        Respondent_age,
        Weight,
        Question_no,
        ID
      ),
      ~as_factor(.x) # Convert remaining variables to factors
    )
  ) |>
  
  # Additional cleaning and standardization
  mutate(
    Respondent_age = case_when(
      is.na(Respondent_age) ~ NA, # Keep missing values as NA
      Respondent_age == "99+" ~ 100, # Convert "99+" to a numeric value
      .default = as.integer(Respondent_age) # Convert other values to integers
    ),
    Has_an_account_at_a_financial_institution = case_match(
      Has_an_account_at_a_financial_institution,
      "yes" ~ "Yes", # Standardize affirmative responses
      "0" ~ "No", # Standardize negative responses
      .default = Has_an_account_at_a_financial_institution
    ),
    Has_an_account = case_match(
      Has_an_account,
      "yes" ~ "Yes", # Same standardization for another variable
      "0" ~ "No",
      .default = Has_an_account
    ),
    Has_a_mobile_money_account = case_match(
      Has_a_mobile_money_account,
      "yes" ~ "Yes", # Standardize affirmative responses
      "0" ~ "No",
      .default = Has_a_mobile_money_account
    )
  )


ERROR: [1m[33mError[39m in `select()`:[22m
[33m![39m Can't select columns that don't exist.
[31m✖[39m Column `Economy` doesn't exist.


In [ ]:
# Rename columns in gfi_2021_tbl using a names dictionary (names_dict_2021).
# The names are dynamically replaced using setNames and !!! operator for splicing.
long_gfi_2021_tbl <- gfi_2021_tbl |> 
  rename(
    !!!setNames(
      names(gfi_2021_tbl), 
      names_dict_2021[names(gfi_2021_tbl)]
    )
  ) |> 
  # Remove unnecessary columns: Economy, Economy_Code, and Gallup_World_Poll_identifier.
  select(-c(Economy, Economy_Code, Gallup_World_Poll_identifier)) |> 
  # Convert the table to a long format for better analysis.
  # Exclude control columns from being pivoted.
  pivot_longer(
    cols = -c(
      !!!syms(control_cols2021) # Dynamically select control columns to exclude from pivoting.
    ),
    names_to = "Question",      # New column name for the variable names being pivoted.
    values_to = "Answers"       # New column name for the values corresponding to variable names.
  ) |> 
  # Mutate to clean and standardize categorical variables using case_match.
  mutate(
    # Map numeric values of Respondent_is_female to meaningful labels.
    Respondent_is_female = case_match(
      Respondent_is_female,
      1 ~ "Female",
      2 ~ "Male",
      .default = NA
    ),
    # Map education level codes to descriptive labels.
    Respondent_education_level = case_match(
      Respondent_education_level,
      1 ~ "completed primary or less",
      2 ~ "secondary",
      3 ~ "completed tertiary or more",
      .default = NA
    ),
    # Map workforce participation status to descriptive labels.
    Respondent_is_in_the_workforce = case_match(
      Respondent_is_in_the_workforce,
      1 ~ "in workforce",
      2 ~ "out of workforce",
      .default = NA
    ),
    # Map income quintiles to descriptive labels for within-economy household income.
    Within_economy_household_income_quintile = case_match(
      Within_economy_household_income_quintile,
      1 ~ "Poorest 20%",
      2 ~ "Second 20%",
      3 ~ "Middle 20%",
      4 ~ "Fourth 20%",
      5 ~ "Richest 20%",
      .default = NA
    ),
    # Map binary responses for financial institution account ownership.
    Has_an_account_at_a_financial_institution = case_match(
      Has_an_account_at_a_financial_institution,
      1 ~ "Yes",
      0 ~ "No",
      .default = NA
    ),
    # Map binary responses for account ownership in general.
    Has_an_account = case_match(
      Has_an_account,
      1 ~ "Yes",
      0 ~ "No",
      .default = NA
    ),
    # Map binary responses for mobile money account ownership.
    Has_a_mobile_money_account = case_match(
      Has_a_mobile_money_account,
      1 ~ "Yes",
      0 ~ "No",
      .default = NA
    )
  ) |> 
  # Group by respondent ID to create a unique question number for each response.
  group_by(ID) |> 
  mutate(
    Question_no = row_number() # Assign sequential question numbers within each group.
  ) |> 
  ungroup() |> 
  # Convert all columns, except specified ones, to factors for easier analysis.
  mutate(
    across(
      -c(ID, Respondent_age, Weight, Question_no), # Exclude columns that should not be converted.
      ~as_factor(.x) # Convert to factor type.
    )
  )


# Models

## Paremtric without Bootsrtap

In [ ]:
weighted_sumw <- combined_tbl |> 
  mutate(
    Within_economy_household_income_quintile  = factor(Within_economy_household_income_quintile, levels = c("Poorest 20%","Second 20%","Middle 20%","Fourth 20%","Richest 20%"),ordered = T),
    Respondent_education_level = factor(Respondent_education_level, levels = c("completed primary or less","secondary","completed tertiary or more"),ordered = T)
)
 workflow() |> 
  add_model(
    logistic_reg() |> set_engine("glm") |> set_mode("classification")
  ) |> 
  add_recipe(
    recipe(Respondent_is_in_the_workforce ~ 
      Respondent_is_female + 
      Respondent_age+
      Has_an_account_at_a_financial_institution+
      Has_a_mobile_money_account+
      Within_economy_household_income_quintile+
           year+
      Respondent_education_level,
     data = weighted_sumw,
          weights = weight) |> 
      step_dummy(c(year,Respondent_is_female,Has_an_account_at_a_financial_institution,Has_a_mobile_money_account)) |> 
      step_ordinalscore(c(Within_economy_household_income_quintile,Respondent_education_level)) |> 
      step_interact(~.:starts_with("year"))
  ) |> 
fit(data = weighted_sumw) |> 
   extract_fit_parsnip() |> 
   tidy()

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-0.630387769,5.791085e-02,-1.088549e+01,1.351747e-27
Respondent_age,-0.007826444,6.264152e-04,-1.249402e+01,8.048189e-36
Within_economy_household_income_quintile,-0.030123903,6.809100e-03,-4.424065e+00,9.686082e-06
Respondent_education_level,0.559857790,1.479642e-02,3.783738e+01,0.000000e+00
year_X2021,-18.935680750,2.744874e+02,-6.898561e-02,9.450011e-01
Respondent_is_female_Female,-1.696339211,1.908269e-02,-8.889412e+01,0.000000e+00
Has_an_account_at_a_financial_institution_Yes,0.749857829,4.148945e-02,1.807346e+01,5.157652e-73
Has_a_mobile_money_account_Yes,0.461204722,2.173457e-02,2.121987e+01,6.259683e-100
Respondent_age_x_year_X2021,0.007826444,3.088155e+00,2.534343e-03,9.979779e-01
